# Summary of work for progress report 2
Sonia Cromp (snc40@pitt.edu)

23 March 2021

I don't have a lot of new code to show for this progress report since a lot of the effort lied in learning how to use CRC and how to set up the networking right (then, days monitoring the data collection). However, I now have the full dataset and can explore some statistics on that a little. I'll also walk through how I set up the networking stuff.

In [14]:
import wikipedia
import os

## The dataset
Languages included, filesize, chunks per language, extracted vs texts vs chunked

# The networking
I use the `getpass` library to securely get the password and the `paramiko` library to establish sftp connections. Although scp is faster, I chose sftp since `paramiko` has support for passing the remote computer's password as an argument, rather than forcing a human to be there to type in the password every time. Instead, my code is set up so that the password is gathered securely, one time at the very start of the script, and stored for repeated use.

---
### Outline of the process

Outside script

1. Get an internet address (or technically, publically expose port 22) for the storage laptop by running `./ngrok tcp 22` in shell and leaving it running
2. On my normal laptop, connect to Pitt's VPN (required to access CRC)
2. ssh to crc.h2p.pitt.edu to access CRC
3. Pass storage laptop's username, address and port to the script as command line arguments

Inside script

1. Prompt user for password, store it for future use in the runtime
2. Establish a test sftp connection with storage laptop to verify password (if fails, prompt user again)
3. After downloading a language's raw dump and extracting XML, send backup to device by opening a new sftp connection, transferring file, then closing connection
4. Transfer another copy of the data after passing through the cleaning steps
5. Transfer another copy of the data after the chunking step
---
What follows is an example that creates a very small exammple helloworld.txt file, then transfers it to another computer. I didn't bother with the VPN or ngrok for this simple example.

In [2]:
import paramiko
import getpass

In [12]:
fname = 'helloworld.txt'
f = open(fname, 'w')
f.writelines('hello world\n')
f.close()

In [8]:
username = 'snc'
port = 22
address = '10.0.0.25' # this is the computer's PRIVATE IP address: the address your internet
# router uses to distinguish this computer from any other device on your own network so 
# you don't recieve all the webpages requested by your brother and grandma or vice versa

# to enable this sftp connection for a mac, go to System Preferences > Sharing > 
# Enable "Remote Login". There, it will display the IP address to use. Your source/local
# and destination/remote computers must be on the same wifi network unless you learn to use
# ngrok or something similar. Don't know how to do it on other operating systems 
# but there's probably tutorials on line if it isn't enabled by defailt.

In [9]:
pwd = getpass.getpass(prompt='sftp password: ') # password for your username 
# on destination/remote computer

sftp password: ········


In [13]:
# ESTABLISH CONNECTION
client = paramiko.client.SSHClient()
client.load_system_host_keys() # this loads any local ssh keys
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(address, port=port, username=username, password=pwd)
sftp = client.open_sftp() # type SFTPClient

# TRANSFER THE FILE
sftp.put('./'+fname, fname) #src, dest path/filename
print('\ttransferred', fname)

# CLOSE THE CONNECTION
client.close()

	transferred ex.txt


If you run this yourself, look in your home (~) directory for the transferred file at the destination/remote computer.